In [1]:
import os, glob, json, csv, subprocess, sys, re
from git import *
from subprocess import Popen, PIPE
from os import path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as ply
import plotly
from plotly.offline import plot, init_notebook_mode, iplot

%matplotlib inline

# Defining repository

In [2]:
userhome = os.path.expanduser('~')
repository = os.path.dirname(userhome + r'/historage-mining/historage/hbase/')

# Defining function to execute git command

In [101]:
def execute_command(cmd, work_dir):
    """Executes a shell command in a subprocess, waiting until it has completed.
 
    :param cmd: Command to execute.
    :param work_dir: Working directory path.
    """
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

# Comparing files with bugs introducing based on the number of line changed

Finding files and their paths with different number of line changed

In [87]:
colnames = ['bug_ref','commitID_1','commitID_2','filename','#line_changed-hist','#line_changed-myers','#line_changed-min','#line_changed-pat']

In [269]:
#Show the csv file with different value of number of line changed between diff-algorithms
data = pd.read_csv('diffbugs_file_differentvalue.csv')
data = data[colnames]
data

,bug_ref,commitID_1,commitID_2,filename,#line_changed-hist,#line_changed-myers,#line_changed-min,#line_changed-pat
0,HBASE-10569,0e5d21e96ae24e8c658a1b74843b72bddcb7435,7716a3ca94,.../hadoop/hbase/master/ActiveMasterManager.java,31,29,29,31
1,HBASE-13686,587b0b4f20bdc0415b6541023e611b69c87dba15,9f43a3bea6,.../apache/hadoop/hbase/quotas/RateLimiter.java,103,97,97,99
2,HBASE-13686,587b0b4f20bdc0415b6541023e611b69c87dba15,9f43a3bea6,.../hadoop/hbase/quotas/TestRateLimiter.java,114,108,108,108
3,HBASE-10569,0e5d21e96ae24e8c658a1b74843b72bddcb7435,7716a3ca94,.../org/apache/hadoop/hbase/master/HMaster.java,1993,1977,1969,1985
4,HBASE-10569,0e5d21e96ae24e8c658a1b74843b72bddcb7435,7716a3ca94,.../hadoop/hbase/regionserver/HRegionServer.java,2285,2425,2285,2285


In [270]:
filename = []
for jj in range(0,len(data)):
    filename.append(((data.iloc[jj][3].split('/'))[-1:])[0])
filename

['ActiveMasterManager.java',
 'RateLimiter.java',
 'TestRateLimiter.java',
 'HMaster.java',
 'HRegionServer.java']

In [271]:
filepath = []
for ii in range(0,len(filename)):
    for root, dirs, files in os.walk(repository):
        for name in files:
            if name == filename[ii]:
                filepath.append(os.path.abspath(os.path.join(root, name)))
#filepath

In [287]:
filepath

['/Users/YusufNugroho/historage-mining/historage/hbase/hbase-server/src/main/java/org/apache/hadoop/hbase/master/ActiveMasterManager.java',
 '/Users/YusufNugroho/historage-mining/historage/hbase/hbase-server/src/main/java/org/apache/hadoop/hbase/quotas/RateLimiter.java',
 '/Users/YusufNugroho/historage-mining/historage/hbase/hbase-server/src/test/java/org/apache/hadoop/hbase/quotas/TestRateLimiter.java',
 '/Users/YusufNugroho/historage-mining/historage/hbase/hbase-server/src/main/java/org/apache/hadoop/hbase/master/HMaster.java',
 '/Users/YusufNugroho/historage-mining/historage/hbase/hbase-server/src/main/java/org/apache/hadoop/hbase/regionserver/HRegionServer.java']

In [272]:
#replacing filename in csv with filepath
for n, row in enumerate(data['filename']):
    data['filename'].replace({data['filename'][n]:filepath[n]},inplace=True)

data.to_csv("diffbugs_file_differentvalue_withfilepath.csv")

In [273]:
cols = ['bug_link','commitID','parenthash']
dtparent = pd.read_csv("parenthash.csv")
dtparent = dtparent[cols]
dtparent

,bug_link,commitID,parenthash
0,HBASE-9426,c079ba4660,c7a72479b6f06a1ab6de66d5d95257f87bb44500
1,HBASE-5209,145d25dfc2,d69299dfdc44a09c7ac2a6a2c4bfffae4a49a327
2,HBASE-15467,bdb0cc8808,65a8d77433cf72ce67e8b5e2efb35f2c2603c349
3,HBASE-13686,9f43a3bea6,587b0b4f20bdc0415b6541023e611b69c87dba15
4,HBASE-10598,c6236a57a5,e09d637d7a61a6e636905fa19495fb145c548d81
5,HBASE-10569,4487333e5e,e44d0d8083e5d4087f1c3a32058daa779c1d3fbb
6,HBASE-10569,d59d9b48f4,7716a3ca949670f0a88472cbb07e2c63e5a50c35
7,HBASE-10569,7716a3ca94,0e5d21e96ae24e8c658a1b74843b72bddcb7435b


# Finding different line changes in files

In [274]:
diff_algorithm = ['myers','histogram','minimal','patience']

In [275]:
files = []
for n, algo in enumerate(diff_algorithm):
    f = []
    for aa in range(0,len(data)):
        diff_cmd = "git diff -w --diff-algorithm=" + algo + " " + data.iloc[aa][1] + " " + data.iloc[aa][2] + " " + filepath[aa] + " > /Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/file-diff/" + filename[aa] + "-" + data.iloc[aa][0] + "_" + algo + "changes.diff"
        execute_command(diff_cmd, repository)
        f.append(filename[aa] + "-" + data.iloc[aa][0] + "_" + algo + "changes.diff")
    files.append(f)

In [276]:
files

[['ActiveMasterManager.java-HBASE-10569_myerschanges.diff',
  'RateLimiter.java-HBASE-13686_myerschanges.diff',
  'TestRateLimiter.java-HBASE-13686_myerschanges.diff',
  'HMaster.java-HBASE-10569_myerschanges.diff',
  'HRegionServer.java-HBASE-10569_myerschanges.diff'],
 ['ActiveMasterManager.java-HBASE-10569_histogramchanges.diff',
  'RateLimiter.java-HBASE-13686_histogramchanges.diff',
  'TestRateLimiter.java-HBASE-13686_histogramchanges.diff',
  'HMaster.java-HBASE-10569_histogramchanges.diff',
  'HRegionServer.java-HBASE-10569_histogramchanges.diff'],
 ['ActiveMasterManager.java-HBASE-10569_minimalchanges.diff',
  'RateLimiter.java-HBASE-13686_minimalchanges.diff',
  'TestRateLimiter.java-HBASE-13686_minimalchanges.diff',
  'HMaster.java-HBASE-10569_minimalchanges.diff',
  'HRegionServer.java-HBASE-10569_minimalchanges.diff'],
 ['ActiveMasterManager.java-HBASE-10569_patiencechanges.diff',
  'RateLimiter.java-HBASE-13686_patiencechanges.diff',
  'TestRateLimiter.java-HBASE-13686_pat

In [277]:
file_to_compare = []
for x in range(0,len(filename)):
    fc = []
    for n,file in enumerate(files):
        fc.append(file[x])
    file_to_compare.append(fc)

In [278]:
file_to_compare[0]

['ActiveMasterManager.java-HBASE-10569_myerschanges.diff',
 'ActiveMasterManager.java-HBASE-10569_histogramchanges.diff',
 'ActiveMasterManager.java-HBASE-10569_minimalchanges.diff',
 'ActiveMasterManager.java-HBASE-10569_patiencechanges.diff']

# Define function to find different line in files

In [279]:
def get_clean_line(line):
    return [entry.strip().lower() for entry in line if entry]

In [280]:
path = "/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/file-diff/"
path2 = path + "diff-line-of-files/"

In [281]:
fn = []
for xx, fc in enumerate(file_to_compare):
    diff_of_diff = []
    with open(path + fc[0]) as first_file, open(path + fc[1]) as second_file, open(path + fc[2]) as third_file, open(path + fc[3]) as fourth_file:
        first_reader = csv.reader(first_file)
        second_reader = csv.reader(second_file)
        third_reader = csv.reader(third_file)
        fourth_reader = csv.reader(fourth_file)

        for first_line, second_line, third_line, fourth_line in zip(first_reader, second_reader, third_reader, fourth_reader):
            first_list, second_list, third_list, fourth_list = get_clean_line(first_line), get_clean_line(second_line), get_clean_line(third_line), get_clean_line(fourth_line)

            if (len(first_list) != len(second_list) or sorted(first_list) != sorted(second_list)) or (len(first_list) != len(third_list) or sorted(first_list) != sorted(third_list)) or (len(first_list) != len(fourth_list) or sorted(first_list) != sorted(fourth_list)) or (len(second_list) != len(third_list) or sorted(second_list) != sorted(third_list)) or (len(second_list) != len(fourth_list) or sorted(second_list) != sorted(fourth_list)) or (len(third_list) != len(fourth_list) or sorted(third_list) != sorted(fourth_list)):
                temp = str('{} :=: {} :=: {} :=: {} :=: {}'.format(
                    first_reader.line_num, first_list, second_list, third_list, fourth_list)).replace("['",'').replace("']",'').split(":=:")
                diff_of_diff.append(temp)
    
    with open (path2 + "diff_of_diff_" + filename[xx] + ".csv", 'w') as csvfile:
        cols = ['line_no','diffmyers-' + filename[xx],'diffhistogram-' + filename[xx],'diffminimal-' + filename[xx],'diffpatience-' + filename[xx]]
        writers = csv.writer(csvfile, delimiter=',')
        writers.writerow(cols)
        for line in diff_of_diff:
            writers.writerow(line)
    
    fn.append(path2 + "diff_of_diff_" + filename[xx] + ".csv")

In [282]:
filepath

['/Users/YusufNugroho/historage-mining/historage/hbase/hbase-server/src/main/java/org/apache/hadoop/hbase/master/ActiveMasterManager.java',
 '/Users/YusufNugroho/historage-mining/historage/hbase/hbase-server/src/main/java/org/apache/hadoop/hbase/quotas/RateLimiter.java',
 '/Users/YusufNugroho/historage-mining/historage/hbase/hbase-server/src/test/java/org/apache/hadoop/hbase/quotas/TestRateLimiter.java',
 '/Users/YusufNugroho/historage-mining/historage/hbase/hbase-server/src/main/java/org/apache/hadoop/hbase/master/HMaster.java',
 '/Users/YusufNugroho/historage-mining/historage/hbase/hbase-server/src/main/java/org/apache/hadoop/hbase/regionserver/HRegionServer.java']

In [283]:
fn

['/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/file-diff/diff-line-of-files/diff_of_diff_ActiveMasterManager.java.csv',
 '/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/file-diff/diff-line-of-files/diff_of_diff_RateLimiter.java.csv',
 '/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/file-diff/diff-line-of-files/diff_of_diff_TestRateLimiter.java.csv',
 '/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/file-diff/diff-line-of-files/diff_of_diff_HMaster.java.csv',
 '/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/file-diff/diff-line-of-files/diff_of_diff_HRegionServer.java.csv']

In [284]:
df = pd.read_csv(fn[4])
df

,line_no,diffmyers-HRegionServer.java,diffhistogram-HRegionServer.java,diffminimal-HRegionServer.java,diffpatience-HRegionServer.java
0,1348,"@@ -2803', '1679 +2505', '100 @@ public class...","@@ -2803', '930 +2505', '6 @@ public class hr...","@@ -2803', '930 +2505', '6 @@ public class hr...","@@ -2803', '930 +2505', '6 @@ public class hr..."
1,1364,+ @override,- },- },- }
2,1365,+ public void updateregionfavorednodesmappin...,-,-,-
3,1366,+ list<org.apache.hadoop.hbase.protobuf....,- // start client methods,- // start client methods,- // start client methods
4,1367,+ inetsocketaddress[] addr = new inetsocke...,-,-,-
5,1368,+ // refer to the comment on the declarati...,- /**,- /**,- /**
6,1369,+ // it is a map of region name to inetsoc...,- * get data from a table.,- * get data from a table.,- * get data from a table.
7,1370,+ for (int i = 0; i < favorednodes.size();...,- *,- *,- *
8,1371,+ addr[i] = inetsocketaddress.createunre...,- * @param controller the rpc controller,- * @param controller the rpc controller,- * @param controller the rpc controller
9,1372,+ favorednodes.get(i).getport());,- * @param request the get request,- * @param request the get request,- * @param request the get request


# Finding the buggy line of files

In [285]:
buggylinefiles = []
for n, algo in enumerate(diff_algorithm):
    bf = []
    for aa in range(0,len(data)):
        diff_cmd = "git diff -U0 -w --diff-algorithm=" + algo + " " + data.iloc[aa][1] + " " + data.iloc[aa][2] + " " + filepath[aa] + " | grep '^[-]' | grep -Ev '^(--- a/|\+\+\+ b/)' > /Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/file-diff/buggylines/" + filename[aa] + "-" + data.iloc[aa][0] + "_" + algo + "buglines.diff"
        execute_command(diff_cmd, repository)
        bf.append(filename[aa] + "-" + data.iloc[aa][0] + "_" + algo + "buglines.diff")
    buggylinefiles.append(bf)

In [286]:
buggylinefiles

[['ActiveMasterManager.java-HBASE-10569_myersbuglines.diff',
  'RateLimiter.java-HBASE-13686_myersbuglines.diff',
  'TestRateLimiter.java-HBASE-13686_myersbuglines.diff',
  'HMaster.java-HBASE-10569_myersbuglines.diff',
  'HRegionServer.java-HBASE-10569_myersbuglines.diff'],
 ['ActiveMasterManager.java-HBASE-10569_histogrambuglines.diff',
  'RateLimiter.java-HBASE-13686_histogrambuglines.diff',
  'TestRateLimiter.java-HBASE-13686_histogrambuglines.diff',
  'HMaster.java-HBASE-10569_histogrambuglines.diff',
  'HRegionServer.java-HBASE-10569_histogrambuglines.diff'],
 ['ActiveMasterManager.java-HBASE-10569_minimalbuglines.diff',
  'RateLimiter.java-HBASE-13686_minimalbuglines.diff',
  'TestRateLimiter.java-HBASE-13686_minimalbuglines.diff',
  'HMaster.java-HBASE-10569_minimalbuglines.diff',
  'HRegionServer.java-HBASE-10569_minimalbuglines.diff'],
 ['ActiveMasterManager.java-HBASE-10569_patiencebuglines.diff',
  'RateLimiter.java-HBASE-13686_patiencebuglines.diff',
  'TestRateLimiter.jav